In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import argparse
import importlib
import data

from glob import glob
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

importlib.reload(data)

In [ ]:
#load img
#to load whole dataset will need to write a torch Dataset class

import data; importlib.reload(data)
batch_size=8

datamodule=data.IRAVENDataModule(batch_size=batch_size)
datamodule.setup()
train_dataloader,val_dataloader,test_dataloader=datamodule.train_dataloader(),datamodule.val_dataloader(),datamodule.test_dataloader()

print(train_dataloader.dataset.__len__(),test_dataloader.dataset.__len__(),val_dataloader.dataset.__len__())

datamodule=data.IRAVENDataModule(batch_size=batch_size,split=(90,10,0))
datamodule.setup()
train_dataloader,val_dataloader,test_dataloader=datamodule.train_dataloader(),datamodule.val_dataloader(),datamodule.test_dataloader()

print(train_dataloader.dataset.__len__(),test_dataloader.dataset.__len__(),val_dataloader.dataset.__len__())


In [ ]:
import collections

train_dataset_fnames=train_dataloader.dataset.filenames
test_dataset_fnames=test_dataloader.dataset.filenames


def get_type_strs(structure):
    types=structure[3]
    types_str=eval(str(types)[1:])
    return types_str

types_str_dataset=[]
for file in test_dataset_fnames:
    structure=np.load(file)['structure']
    types_str_dataset.append(get_type_strs(structure))
    






    frequency=collections.Counter(types_str_dataset)


In [ ]:
t1=torch.full((5,5),4.0,requires_grad=True)
t2=torch.full((5,5),2.0,requires_grad=False)
t3=t1*t2
print(t3.requires_grad)

## Testing SCL model-mask divergence

In [ ]:
#testing masked scl

import masking,data,utils,models.SCL_model
from data import IRAVENDataModule
importlib.reload(masking)
importlib.reload(data)
importlib.reload(utils)
importlib.reload(models.SCL_model)

from models.SCL_model import SCLTrainingWrapper,SCL

#setup
train=False
test=True
logging=True
model_ckpt='/Users/iyngkarrankumar/Documents/AI/AVR-functional-modularity/SCL_pretrain_80.ckpt'
task_path='datasets/light'
save_freq= 10000
batch_size=8
split=(85,15,0)
device=torch.device('cpu')

#dataset setup
if 1:
    #task dataset
    path=task_path
    data_module=IRAVENDataModule(batch_size=batch_size,split=split)
    data_module.prepare_data()
    data_module.setup(root_dir=path)
    train_dataloader_task,test_dataloader_task=data_module.train_dataloader(),data_module.test_dataloader()
    x,y,*rest=next(iter(train_dataloader_task))

    #NOT task dataset
    path_='datasets/originals_masking'
    data_module_=IRAVENDataModule(batch_size=batch_size,split=split)
    data_module_.prepare_data()
    data_module_.setup(root_dir=path_)
    test_dataloader_not_task=data_module_.test_dataloader()

#setup model
if 1:
    scl_kwargs={
        "image_size":160,                            # size of image
        "set_size": 9,                               # number of questions + 1 answer
        "conv_channels": [1, 16, 16, 32, 32, 32],    # convolutional channel progression, 1 for greyscale, 3 for rgb
        "conv_output_dim": 80,                       # model dimension, the output dimension of the vision net
        "attr_heads": 10,                            # number of attribute heads
        "attr_net_hidden_dims": [128],               # attribute scatter transform MLP hidden dimension(s)
        "rel_heads": 80,                             # number of relationship heads
        "rel_net_hidden_dims": [64, 32, 5] 
    }
    model=SCL(**scl_kwargs)

    #load
    state_dict=utils.get_SCL_state_dict(model_ckpt)
    model.load_state_dict(state_dict)
    model.eval()
    print(f'Model training: {model.training}')

#initialise masked model
if 1: 
    init_kwargs={
        'model':model,
        'train_dataloader':train_dataloader_task,
        'test_dataloader1':test_dataloader_task,
        'test_dataloader2':test_dataloader_not_task,
        'device':device,
        'savedir':'model_ckpts/FFN',
    }

    masked_scl=masking.MaskedSCLModel(init_kwargs)

#train
if 1:
    train_kwargs={
        'alpha':1e-6,
        'n_epochs':2 if test else 10,
        'n_batches':20 if test else 'full',
        'val_every_n_steps':5 if test else 10,
        'eval_every_n_steps':1e10,
        'n_val_batches':2 if test else 100,
        'n_eval_batches':2 if test else 100,
        'save_freq':save_freq,
        'logging':logging,

        }

    if train:
        masked_scl.train(**train_kwargs)


In [ ]:
#testing masked scl

import masking,data,utils,models.SCL_model
from data import IRAVENDataModule
importlib.reload(masking)
importlib.reload(data)
importlib.reload(utils)
importlib.reload(models.SCL_model)

from models.SCL_model import SCLTrainingWrapper,SCL

#setup
train=False
test=True
logging=True
model_ckpt='/Users/iyngkarrankumar/Documents/AI/AVR-functional-modularity/SCL_pretrain_80.ckpt'
task_path='datasets/light'
save_freq= 10000
batch_size=8
split=(85,15,0)
device=torch.device('cpu')

#dataset setup
if 1:
    #task dataset
    path=task_path
    data_module=IRAVENDataModule(batch_size=batch_size,split=split)
    data_module.prepare_data()
    data_module.setup(root_dir=path)
    train_dataloader_task,test_dataloader_task=data_module.train_dataloader(),data_module.test_dataloader()
    x,y,*rest=next(iter(train_dataloader_task))

    #NOT task dataset
    path_='datasets/originals_masking'
    data_module_=IRAVENDataModule(batch_size=batch_size,split=split)
    data_module_.prepare_data()
    data_module_.setup(root_dir=path_)
    test_dataloader_not_task=data_module_.test_dataloader()

#setup model
if 1:
    scl_kwargs={
        "image_size":160,                            # size of image
        "set_size": 9,                               # number of questions + 1 answer
        "conv_channels": [1, 16, 16, 32, 32, 32],    # convolutional channel progression, 1 for greyscale, 3 for rgb
        "conv_output_dim": 80,                       # model dimension, the output dimension of the vision net
        "attr_heads": 10,                            # number of attribute heads
        "attr_net_hidden_dims": [128],               # attribute scatter transform MLP hidden dimension(s)
        "rel_heads": 80,                             # number of relationship heads
        "rel_net_hidden_dims": [64, 32, 5] 
    }
    model=SCL(**scl_kwargs)

    #load
    state_dict=utils.get_SCL_state_dict(model_ckpt)
    model.load_state_dict(state_dict)
    model.eval()
    print(f'Model training: {model.training}')

#initialise masked model
if 1: 
    init_kwargs={
        'model':model,
        'train_dataloader':train_dataloader_task,
        'test_dataloader1':test_dataloader_task,
        'test_dataloader2':test_dataloader_not_task,
        'device':device,
        'savedir':'model_ckpts/FFN',
    }

    masked_scl=masking.MaskedSCLModel(init_kwargs)

#train
if 1:
    train_kwargs={
        'alpha':1e-6,
        'n_epochs':2 if test else 10,
        'n_batches':20 if test else 'full',
        'val_every_n_steps':5 if test else 10,
        'eval_every_n_steps':1e10,
        'n_val_batches':2 if test else 100,
        'n_eval_batches':2 if test else 100,
        'save_freq':save_freq,
        'logging':logging,

        }

    if train:
        masked_scl.train(**train_kwargs)


In [ ]:
import utils
importlib.reload(utils)
task_path='datasets/originals'



data_module=IRAVENDataModule(batch_size=batch_size)
data_module.prepare_data()
data_module.setup(root_dir=task_path)
train_dataloader_task,test_dataloader_task=data_module.train_dataloader(),data_module.test_dataloader()
x,y,*rest=next(iter(train_dataloader_task))
x,y=x.to(device),y.to(device)

#compare accuracies
if 1:
    accuracies=[]
    accuracies_masked=[]
    n_batches=10


    for batch_idx,batch in enumerate(test_dataloader_task):
        print(batch_idx)
        if batch_idx==n_batches:
            break
        x,y,*rest=batch
        x,y=x.to(device),y.to(device)
        
        x_=utils.prepare_input_SCL(x)
        model.eval()
        y_hat=model(x_)
        acc=utils.calculate_accuracy(y_hat,y)
        accuracies.append(acc)
        
        y_hat_masked=masked_scl.forward(x)
        acc_masked=utils.calculate_accuracy(y_hat_masked,y)
        accuracies_masked.append(acc_masked)
        
        
    print(np.mean(accuracies),np.mean(accuracies_masked))

#calculate model feature differences
if 0:

    x_=utils.prepare_input_SCL(x)
    model.eval() #model uses batchnorm, so put in eval
    y_hat=model(x_)

    y_hat_masked=masked_scl.forward(x)

    model_features=model.layer_outputs
    masked_features=masked_scl.layer_outputs
    mse_difference=[]
    names=['input','post-vision','post-attr','post-rels','post-logits']


    for idx,name in enumerate(names):
        mse_difference.append((name,F.mse_loss(model_features[idx],masked_features[idx]).item()))
    print(mse_difference)

if 0:
    x_=utils.prepare_input_SCL(x)
    x_in=copy.deepcopy(x_)
    named_children=masking.get_named_children(model)
    named_vision_modules={k:v for k,v in named_children.items() if 'vision' in k}
    names=list(named_vision_modules.keys());names=['in']+names
    vision_outputs=[]

    for idx,(name,module) in enumerate(named_vision_modules.items()):
        if idx==0:
            b, m, n, c, h, w = x_in.shape
            x_in = x_in.view(-1, c, h, w)
            vision_outputs.append(x_in)
            out=module(x_in)
            vision_outputs.append(out)
        else:
            out=module(out)
            vision_outputs.append(out)


    out = masked_scl.forward(x)

    mse_differences=[]
    for (name,(t1,t2)) in zip(names,zip(vision_outputs,masked_scl.vision_outputs)):
        diff=F.mse_loss(t1,t2).item()
        mse_differences.append((name,diff))
    print(mse_differences)
    print(f'Mask sparsity: {utils.sparsity((masked_model.transform_logit_tensors()).values())}')


if 0:

    _=masked_scl.forward(x) #initialise binaries 
    params=masked_scl.param_dict
    binaries=masked_scl.binaries
    print(f'Sparsity: {utils.sparsity(binaries.values())}')

    param_differences=[]

    for name in named_vision_modules.keys():
        try: 
            weight,bias=named_vision_modules[name].weight,named_vision_modules[name].bias
            weight_,bias_=params[name+'.weight']*binaries[name+'.weight'],params[name+'.bias']*binaries[name+'.bias']
            param_differences.append((name,F.mse_loss(weight,weight_).item(),F.mse_loss(bias,bias_).item()))
        except:
            #dealing with Flatten layer
            pass 


    print(param_differences)

